# 📈 Notebook 04: Deep Evaluation

**Predicting F1 Race Finishing Positions Using Practice & Qualifying Data**

Final evaluation of the best model on the 2025 test set.

1. Overall test performance
2. Per-race accuracy breakdown
3. Per-driver analysis
4. Podium prediction accuracy
5. Error distribution
6. Findings & limitations

In [1]:
import sys, os, warnings
warnings.filterwarnings('ignore')
sys.path.insert(0, os.path.abspath('..'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr

import config
from src.feature_engineering import load_feature_matrix
from src.model import (
    train_and_evaluate,
    evaluate_model,
    compute_spearman_per_race,
)
from src.utils import (
    plot_predicted_vs_actual,
    plot_per_race_mae,
    plot_feature_importance,
)

sns.set_theme(style='whitegrid')
%matplotlib inline

## 1. Run Pipeline & Get Predictions

In [2]:
# Full pipeline
pipeline_results = train_and_evaluate(tune=True)

results = pipeline_results['results']
models = pipeline_results['models']
predictions = pipeline_results['predictions']
fi = pipeline_results['feature_importance']

# Choose best model
best_model_key = min(results, key=lambda k: results[k]['mae'])
print(f"\n✅ Best model: {best_model_key}")

test_df = predictions['test_df'].copy()
best_pred_key = 'y_pred_rf' if best_model_key == 'random_forest' else 'y_pred_xgb'
y_pred = np.clip(predictions[best_pred_key], 1, 20)
y_true = test_df['race_position'].values
test_df['predicted_position'] = y_pred


Training set: 918 samples ([2023, 2024])
Test set: 479 samples ([2025])

── Training Baseline ──
  MAE: 3.411, Spearman: 0.652

── Training Random Forest ──
  Tuning Random Forest...
  Best RF params: {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': None, 'max_depth': 6}
  MAE: 3.302, Spearman: 0.651

── Training XGBoost ──
  Tuning XGBoost...
  Best XGB params: {'subsample': 0.7, 'reg_lambda': 5.0, 'reg_alpha': 1.0, 'n_estimators': 500, 'min_child_weight': 5, 'max_depth': 6, 'learning_rate': 0.01, 'colsample_bytree': 0.7}
  MAE: 3.302, Spearman: 0.643

MODEL COMPARISON (Test Set - 2025)
                                  model    mae   rmse spearman_rho spearman_p per_race_spearman top3_accuracy ndcg_10
baseline       Baseline (Grid Position)  3.411  4.367        0.652        0.0             0.651          0.75   0.959
random_forest             Random Forest  3.302  4.293        0.651        0.0             0.648          0.75   0.833
xgboost       

## 2. Overall Test Performance

In [ ]:
# Overall metrics
best_results = results[best_model_key]
print(f"\n{'='*60}")
print(f"OVERALL TEST PERFORMANCE ({best_model_key.replace('_', ' ').upper()})")
print(f"{'='*60}")
for metric, value in best_results.items():
    if metric != 'model':
        print(f"  {metric:>20s}: {value}")

In [ ]:
# Predicted vs Actual
fig = plot_predicted_vs_actual(y_true, y_pred, model_name=best_model_key.replace('_', ' ').title())
plt.show()

## 3. Per-Race Analysis

In [ ]:
# Per-race MAE
fig = plot_per_race_mae(test_df, y_pred, model_name=best_model_key.replace('_', ' ').title())
plt.show()

In [ ]:
# Per-race Spearman correlation
test_df['abs_error'] = np.abs(test_df['race_position'] - test_df['predicted_position'])

per_race = test_df.groupby(['RoundNumber', 'EventName']).apply(
    lambda g: pd.Series({
        'MAE': g['abs_error'].mean(),
        'Spearman': spearmanr(g['race_position'], g['predicted_position'])[0] if len(g) > 2 else np.nan,
        'Drivers': len(g),
    })
).reset_index().sort_values('RoundNumber')

print("\n── Per-Race Accuracy ──")
display(per_race.round(3))

print(f"\n  Best race:  {per_race.loc[per_race['Spearman'].idxmax(), 'EventName']} (ρ={per_race['Spearman'].max():.3f})")
print(f"  Worst race: {per_race.loc[per_race['Spearman'].idxmin(), 'EventName']} (ρ={per_race['Spearman'].min():.3f})")

## 4. Per-Driver Analysis

In [ ]:
# Per-driver accuracy
per_driver = test_df.groupby('Driver').apply(
    lambda g: pd.Series({
        'Races': len(g),
        'Avg_Actual': g['race_position'].mean(),
        'Avg_Predicted': g['predicted_position'].mean(),
        'MAE': g['abs_error'].mean(),
        'Max_Error': g['abs_error'].max(),
    })
).sort_values('MAE')

print("\n── Per-Driver Accuracy (sorted by MAE) ──")
display(per_driver.round(2))

In [ ]:
# Driver accuracy visualization
fig, ax = plt.subplots(figsize=(14, 6))
colors = sns.color_palette('YlOrRd', len(per_driver))
bars = ax.barh(per_driver.index, per_driver['MAE'], color=colors, edgecolor='k', linewidth=0.5)
ax.set_xlabel('MAE (positions)')
ax.set_title('Prediction Accuracy per Driver (2025)')
ax.axvline(x=per_driver['MAE'].mean(), color='red', linestyle='--',
           label=f'Mean MAE: {per_driver["MAE"].mean():.2f}')
ax.legend()
plt.tight_layout()
plt.show()

## 5. Podium Prediction Accuracy

In [ ]:
# Podium (top-3) prediction accuracy per race
podium_results = []
for (rnd, name), group in test_df.groupby(['RoundNumber', 'EventName']):
    actual_podium = set(group.nsmallest(3, 'race_position')['Driver'])
    predicted_podium = set(group.nsmallest(3, 'predicted_position')['Driver'])
    overlap = actual_podium & predicted_podium
    podium_results.append({
        'Round': rnd,
        'Race': name,
        'Actual Podium': ', '.join(sorted(actual_podium)),
        'Predicted Podium': ', '.join(sorted(predicted_podium)),
        'Correct': len(overlap),
        'Accuracy': len(overlap) / 3,
    })

podium_df = pd.DataFrame(podium_results).sort_values('Round')
print("\n── Podium Prediction Accuracy ──")
display(podium_df)

print(f"\nOverall Podium Accuracy: {podium_df['Accuracy'].mean():.1%}")
print(f"Perfect Podiums: {(podium_df['Accuracy'] == 1.0).sum()} / {len(podium_df)}")
print(f"At least 2/3: {(podium_df['Accuracy'] >= 2/3).sum()} / {len(podium_df)}")

## 6. Error Distribution

In [ ]:
# Error distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Absolute error histogram
axes[0].hist(test_df['abs_error'], bins=range(0, 16), edgecolor='black',
             alpha=0.7, color='steelblue')
axes[0].set_xlabel('Absolute Error (positions)')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of Prediction Errors')
axes[0].axvline(x=test_df['abs_error'].mean(), color='red', linestyle='--',
                label=f'Mean: {test_df["abs_error"].mean():.2f}')
axes[0].legend()

# Error by actual position
error_by_pos = test_df.groupby('race_position')['abs_error'].mean()
axes[1].bar(error_by_pos.index, error_by_pos.values,
            color=sns.color_palette('coolwarm', len(error_by_pos)),
            edgecolor='k', linewidth=0.5)
axes[1].set_xlabel('Actual Race Position')
axes[1].set_ylabel('Mean Absolute Error')
axes[1].set_title('Error by Actual Position')

plt.tight_layout()
plt.show()

# Error stats
print(f"\nError Statistics:")
print(f"  Within ±1 position: {(test_df['abs_error'] <= 1).sum() / len(test_df):.1%}")
print(f"  Within ±2 positions: {(test_df['abs_error'] <= 2).sum() / len(test_df):.1%}")
print(f"  Within ±3 positions: {(test_df['abs_error'] <= 3).sum() / len(test_df):.1%}")
print(f"  Within ±5 positions: {(test_df['abs_error'] <= 5).sum() / len(test_df):.1%}")

## 7. Findings & Limitations

In [ ]:
print(f"""
{'='*60}
📋 FINAL EVALUATION SUMMARY
{'='*60}

Model: {best_model_key.replace('_', ' ').title()}
Train: 2023-2024 | Test: 2025

KEY METRICS:
  • MAE:            {best_results['mae']:.3f} positions
  • RMSE:           {best_results['rmse']:.3f} positions  
  • Spearman ρ:     {best_results['spearman_rho']:.3f}
  • NDCG@10:        {best_results.get('ndcg_10', 'N/A')}
  • Top-3 Accuracy: {best_results.get('top3_accuracy', 'N/A')}

KEY FINDINGS:
  1. Grid position is the single strongest predictor
  2. Practice data adds marginal but measurable value
  3. Rolling driver/team form captures season momentum
  4. Model struggles most with mid-pack positions (7-14)
  5. Wet races and incidents cause the largest errors

LIMITATIONS:
  - Cannot predict crashes, mechanical failures, or penalties
  - Strategy (pit stops, tire management) not modeled
  - Safety cars and red flags create unpredictable shuffles
  - Sprint format weekends may reduce practice data utility
  - Limited to 2 seasons of training data

FUTURE IMPROVEMENTS:
  - Add tire strategy features (compound choices, pit window)
  - Include historical circuit-specific performance
  - Model DNF probability separately
  - Use telemetry data for deeper practice insights
  - Expand training to more seasons (2020-2024)
""")